In [ ]:
import pandas as pd
import numpy as np
from scipy import spatial
from sklearn.metrics import  jaccard_similarity_score, jaccard_score
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
anime = pd.read_csv("/content/drive/My Drive/2-1/88956262RecommenderSystems/anime.csv")
anime["genre_lower"]=list(anime["genre"].map(lambda x: str(x).lower()))
anime

,anime_id,name,genre,type,episodes,rating,members,genre_lower
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,"drama, romance, school, supernatural"
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,"action, adventure, drama, fantasy, magic, mili..."
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262,"action, comedy, historical, parody, samurai, s..."
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572,"sci-fi, thriller"
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266,"action, comedy, historical, parody, samurai, s..."
...,...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211,hentai
12290,5543,Under World,Hentai,OVA,1,4.28,183,hentai
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219,hentai
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175,hentai


In [ ]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   anime_id     12294 non-null  int64  
 1   name         12294 non-null  object 
 2   genre        12232 non-null  object 
 3   type         12269 non-null  object 
 4   episodes     12294 non-null  object 
 5   rating       12064 non-null  float64
 6   members      12294 non-null  int64  
 7   genre_lower  12294 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 768.5+ KB


In [ ]:
data1 = anime
data_genre = pd.DataFrame(data1['genre_lower'])
sd = data_genre.assign(genre_lower=data_genre.genre_lower.str.split(",")).explode('genre_lower')
sd.groupby(['genre_lower'])
sd['genre_lower'] = [str(x.strip()) for x in sd['genre_lower']]
mylistgenre = list(dict.fromkeys(list(sd['genre_lower'])))
mylistgenre1 = mylistgenre.copy()
mylistgenre1.insert(0,'name')
mylistgenre1.insert(1,'genre_lower')
print(mylistgenre1)
print(mylistgenre)

['name', 'genre_lower', 'drama', 'romance', 'school', 'supernatural', 'action', 'adventure', 'fantasy', 'magic', 'military', 'shounen', 'comedy', 'historical', 'parody', 'samurai', 'sci-fi', 'thriller', 'sports', 'super power', 'space', 'slice of life', 'mecha', 'music', 'mystery', 'seinen', 'martial arts', 'vampire', 'shoujo', 'horror', 'police', 'psychological', 'demons', 'ecchi', 'josei', 'shounen ai', 'game', 'dementia', 'harem', 'cars', 'kids', 'shoujo ai', 'nan', 'hentai', 'yaoi', 'yuri']
['drama', 'romance', 'school', 'supernatural', 'action', 'adventure', 'fantasy', 'magic', 'military', 'shounen', 'comedy', 'historical', 'parody', 'samurai', 'sci-fi', 'thriller', 'sports', 'super power', 'space', 'slice of life', 'mecha', 'music', 'mystery', 'seinen', 'martial arts', 'vampire', 'shoujo', 'horror', 'police', 'psychological', 'demons', 'ecchi', 'josei', 'shounen ai', 'game', 'dementia', 'harem', 'cars', 'kids', 'shoujo ai', 'nan', 'hentai', 'yaoi', 'yuri']


In [ ]:
data2 = anime
#  e_genres = data2['genres']
sum_data=[]
sum_data1=[]
dataf ={}
for ind in data2.index:
  m=data2['genre_lower'][ind]
  m_id=data2['anime_id'][ind]
  ad= m.split(",")
  sds = [str(x.strip()) for x in ad]
  my_ad = [1 if x in sds else 0 for x in list(mylistgenre)]
  my_ad.insert(0,data2['name'][ind])
  my_ad.insert(1,ad)
  sum_data.append(my_ad)
# print(sum_data)
data_t = pd.DataFrame(np.array(sum_data),columns=mylistgenre1, index=data2['anime_id'])
data_t

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


,name,genre_lower,drama,romance,school,supernatural,action,adventure,fantasy,magic,military,shounen,comedy,historical,parody,samurai,sci-fi,thriller,sports,super power,space,slice of life,mecha,music,mystery,seinen,martial arts,vampire,shoujo,horror,police,psychological,demons,ecchi,josei,shounen ai,game,dementia,harem,cars,kids,shoujo ai,nan,hentai,yaoi,yuri
anime_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
32281,Kimi no Na wa.,"[drama, romance, school, supernatural]",1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5114,Fullmetal Alchemist: Brotherhood,"[action, adventure, drama, fantasy, magic,...",1,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
28977,Gintama°,"[action, comedy, historical, parody, samur...",0,0,0,0,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9253,Steins;Gate,"[sci-fi, thriller]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9969,Gintama&#039;,"[action, comedy, historical, parody, samur...",0,0,0,0,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9316,Toushindai My Lover: Minami tai Mecha-Minami,[hentai],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5543,Under World,[hentai],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5621,Violence Gekiga David no Hoshi,[hentai],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
data = data_t.copy()
data.drop(['name','genre_lower'], axis=1, inplace=True)
data = data.astype(int)
data

,drama,romance,school,supernatural,action,adventure,fantasy,magic,military,shounen,comedy,historical,parody,samurai,sci-fi,thriller,sports,super power,space,slice of life,mecha,music,mystery,seinen,martial arts,vampire,shoujo,horror,police,psychological,demons,ecchi,josei,shounen ai,game,dementia,harem,cars,kids,shoujo ai,nan,hentai,yaoi,yuri
anime_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
32281,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5114,1,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
28977,0,0,0,0,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9253,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9969,0,0,0,0,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9316,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5543,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5621,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
sum_=data.loc[[223]].values.tolist()
print(sum_)
# ,32281

[[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [ ]:
sum_1=data.loc[[25389]].values.tolist()
print(sum_1)

[[0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [ ]:
sum_2=data_t.loc[[22777]]
sum_2

,name,genre_lower,drama,romance,school,supernatural,action,adventure,fantasy,magic,military,shounen,comedy,historical,parody,samurai,sci-fi,thriller,sports,super power,space,slice of life,mecha,music,mystery,seinen,martial arts,vampire,shoujo,horror,police,psychological,demons,ecchi,josei,shounen ai,game,dementia,harem,cars,kids,shoujo ai,nan,hentai,yaoi,yuri
anime_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22777,Dragon Ball Kai (2014),"[action, adventure, comedy, fantasy, marti...",0,0,0,0,1,1,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
datac2 = data.values.tolist()
datacSetI = list(sum_)
datac0=[]
for ag in datac2:
  datacSetII = ag
  result = spatial.distance.cosine(datacSetI, datacSetII)
  datac0.append(result)
datac0_df =  pd.DataFrame(np.array(datac0),index=anime['anime_id'],columns=['Cosine'])
datac0_df['name'] = data_t['name']
datac0_df['genre'] = data_t['genre_lower']
datac0_df.reset_index().sort_values(by = ['Cosine'],ascending = True).head(15)

,anime_id,Cosine,name,genre
346,223,0.000000,Dragon Ball,"[adventure, comedy, fantasy, martial arts, ..."
4275,6714,0.074180,Dragon Ball Z: Atsumare! Gokuu World,"[action, adventure, comedy, fantasy, marti..."
4314,987,0.074180,Dragon Ball GT: Goku Gaiden! Yuuki no Akashi w...,"[action, adventure, comedy, fantasy, marti..."
3203,22695,0.074180,Dragon Ball Z: Summer Vacation Special,"[action, adventure, comedy, fantasy, marti..."
588,6033,0.074180,Dragon Ball Kai,"[action, adventure, comedy, fantasy, marti..."
1409,25389,0.074180,Dragon Ball Z Movie 15: Fukkatsu no F,"[action, adventure, comedy, fantasy, marti..."
5999,904,0.074180,Dragon Ball Z Movie 11: Super Senshi Gekiha!! ...,"[action, adventure, comedy, fantasy, marti..."
515,22777,0.074180,Dragon Ball Kai (2014),"[action, adventure, comedy, fantasy, marti..."
1930,30694,0.074180,Dragon Ball Super,"[action, adventure, comedy, fantasy, marti..."
206,813,0.074180,Dragon Ball Z,"[action, adventure, comedy, fantasy, marti..."
